In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
raw_data = pd.read_csv("/Users/schlinkertc/Flatiron/projects/mod4_project/developer_survey_2019/survey_results_public.csv")
raw_data.sort_values('ConvertedComp',axis=0,ascending=False)[:50]

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
25376,25524,I am a developer by profession,Yes,Less than once per year,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers,33.0,Man,No,Straight / Heterosexual,Hispanic or Latino/Latina,NaN,Appropriate in length,Neither easy nor difficult
27847,28008,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area,28.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
84868,85395,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,United States,No,Associate degree,I never declared a major,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
44271,44552,I am a developer by profession,No,Never,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Tech...,26.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Neither easy nor difficult
68771,69211,"I am not primarily a developer, but I write co...",No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",...,Just as welcome now as I felt last year,NaN,23.0,Man,No,Straight / Heterosexual,White or of European descent,No,Too long,Neither easy nor difficult
87814,88362,I am a developer by profession,No,Less than once per year,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Industry news about technologies you're intere...,40.0,Man,No,Straight / Heterosexual,Black or of African descent,No,Appropriate in length,Easy
55110,55459,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Other doctoral degree (Ph.D, Ed.D., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area,42.0,Man,No,Straight / Heterosexual,White or of European descent,Yes,Too long,Neither easy nor difficult
55106,55455,I am a developer by profession,Yes,Never,"OSS is, on average, of LOWER quality than prop...",Employed full-time,United States,"Yes, full-time","Master’s degree (MA, MS, M.Eng., MBA, etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,28.0,Man,No,Straight / Heterosexual,South Asian,No,Too long,Easy
18884,18987,I am a developer by profession,Yes,Once a month or more often,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,52.0,Man,No,Straight / Heterosexual,White or o

In [3]:
raw_data.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [4]:
# for x in list(raw_data.columns):
#     if len(raw_data[x].unique())<8:
#         print(x)

In [5]:
raw_data['SOAccount'].unique()

array(['No', 'Yes', "Not sure / can't remember", nan], dtype=object)

In [6]:
cleaned_df = (raw_data[['Respondent','ConvertedComp', 
                        'WorkWeekHrs','YearsCode','Age',
                        'OrgSize','Gender','Ethnicity',
                        'LanguageWorkedWith','Country',
                        'Extraversion','Dependents','Trans',
                        'EdLevel','Student','MainBranch','WorkLoc','WorkPlan',
                       'ImpSyn','Employment','CodeRevHrs','OpSys','BetterLife',
                        'ResumeUpdate','YearsCodePro','LastHireDate','JobSat',
                        'CareerSat','JobSeek','Hobbyist','CompFreq','SOVisitFreq',
                        'SOPartFreq','FizzBuzz'
                        ]].dropna())


In [7]:
africa = [
    "Algeria", "Angola", "Benin",
    "Botswana", "Burkina Faso", "Burundi", "Cameroon",
    "Cape Verde", "Central African Republic", "Chad", "Comoros",
    "Republic of the Congo", "Democratic Republic of the Congo", "Congo, Republic of the...", "Côte d'Ivoire", "Djibouti",
    "Egypt", "Equatorial Guinea", "Eritrea", "Ethiopia",
    "Gabon", "The Gambia", "Ghana", "Guinea",
    "Guinea-Bissau", "Kenya", "Lesotho", "Liberia",
    "Libya", "Libyan Arab Jamahiriya", "Madagascar", "Malawi", "Mali",
    "Mauritania", "Mauritius", "Morocco", "Mozambique",
    "Namibia", "Niger", "Nigeria", "Rwanda",
    "São Tomé and Príncipe", "Senegal", "Seychelles", "Sierra Leone",
    "Somalia", "South Africa", "South Sudan", "Sudan",
    "Swaziland", "Tanzania", "United Republic of Tanzania", "Togo", "Tunisia",
    "Uganda", "Western Sahara", "Zambia", "Zimbabwe"
]
asia = [
    "Afghanistan", "Armenia", "Azerbaijan", "Bahrain",
    "Bangladesh", "Bhutan", "Brunei", "Brunei Darussalam", "Cambodia",
    "China", "Cyprus", "East Timor", "Timor-Leste", "Georgia",
    "Hong Kong", "Hong Kong (S.A.R.)", "India", "Indonesia", "Iran",
    "Iraq", "Israel", "Japan", "Jordan",
    "Kazakhstan", "Kuwait", "Kyrgyzstan", "Laos", "Lao People's Democratic Republic",
    "Lebanon", "Malaysia", "Maldives", "Mongolia",
    "Myanmar", "Nepal", "North Korea", "Republic of Korea", "Oman",
    "Pakistan", "Palestine", "Philippines", "Qatar",
    "Russia", "Saudi Arabia", "Singapore", "South Korea",
    "Sri Lanka", "Syria", "Syrian Arab Republic", "Taiwan", "Tajikistan",
    "Thailand", "Turkey", "Turkmenistan", "United Arab Emirates",
    "Uzbekistan", "Vietnam", "Viet Nam", "Yemen"
]
europe = [
    "Albania", "Andorra", "Austria", "Belarus",
    "Belgium", "Bosnia and Herzegovina", "Bulgaria", "Croatia",
    "Czech Republic", "Denmark", "Estonia", "Finland",
    "France", "Germany", "Greece", "Hungary",
    "Iceland", "Republic of Ireland", "Ireland", "Italy", "Kosovo ",
    "Latvia", "Liechtenstein", "Lithuania", "Luxembourg",
    "Macedonia", "The former Yugoslav Republic of Macedonia", "Republic of Moldova", "Monaco",
    "Malta", "Moldova", "Montenegro", "Netherlands", "Norway", "Poland",
    "Portugal", "Romania", "Russia", "Russian Federation", "San Marino",
    "Serbia", "Slovakia", "Slovenia", "Spain",
    "Sweden", "Switzerland", "Ukraine", "United Kingdom",
    "Vatican City "
]
north_america = [
    "Antigua and Barbuda", "Anguilla", "Aruba", "The Bahamas",
    "Barbados", "Belize", "Bermuda", "Bonaire",
    "British Virgin Islands", "Canada", "Cayman Islands", "Clipperton Island",
    "Costa Rica", "Cuba", "Curaçao", "Dominica",
    "Dominican Republic", "El Salvador", "Greenland", "Grenada",
    "Guadeloupe", "Guatemala", "Haiti", "Honduras",
    "Jamaica", "Martinique", "Mexico", "Montserrat",
    "Navassa Island", "Nicaragua", "Panama", "Puerto Rico",
    "Saba", "Saint Barthelemy", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Martin", "Saint Pierre and Miquelon", "Saint Vincent and the Grenadines", "Sint Eustatius",
    "Sint Maarten", "Trinidad and Tobago", "Turks and Caicos", "United States",
    "US Virgin Islands"
]
south_america = [
    "Argentina", "Bolivia", "Brazil", "Chile",
    "Colombia", "Ecuador", "French Guiana", "Guyana",
    "Paraguay", "Peru", "South Georgia and the South Sandwich Islands", "Suriname",
    "Uruguay", "Venezuela", "Venezuela, Bolivarian Republic of..."
]
oceania_australia = [
    "Australia", "Federated States of Micronesia", "Fiji", "Kiribati",
    "Marshall Islands", "Nauru", "New Zealand", "Palau",
    "Papua New Guinea", "Samoa", "Solomon Islands", "Tonga",
    "Tuvalu", "Vanuatu"
]
def GetConti(country):
    if country in africa:
        return "Africa"
    elif country in asia:
        return "Asia"
    elif country in europe:
        return "Europe"
    elif country in north_america:
        return "North America"
    elif country in south_america:
        return "South America"
    elif country in oceania_australia:
        return "Oceania Australia"
    else:
        return "None"
cleaned_df['Continent'] = cleaned_df['Country'].apply(lambda x: GetConti(x))
clean_other = cleaned_df[cleaned_df['Continent'] == 'None' ].index
cleaned_df.drop(clean_other, inplace=True)

In [8]:
cleaned_df = cleaned_df.replace(to_replace =["Less than 1 year","More than 50 years"], value =["0","50"])
cleaned_df.loc[cleaned_df['Gender'].str.contains(";"),'Gender']='Non-binary, genderqueer, or gender non-conforming'
cleaned_df['YearsCode'] = pd.to_numeric(cleaned_df['YearsCode'])

cleaned_df['YearsCodePro'] = pd.to_numeric(cleaned_df['YearsCodePro'])

cleaned_df['LanguageWorkedWith'] = [x.split(sep=';') for x in cleaned_df['LanguageWorkedWith'].values]
cleaned_df['Ethnicity'] = [x.split(sep=';') for x in cleaned_df['Ethnicity'].values]

In [9]:
indexNames = cleaned_df[cleaned_df['WorkWeekHrs'] > 100].index
cleaned_df.drop(indexNames , inplace=True)

In [10]:
df_age_calc = pd.DataFrame(cleaned_df[['Respondent','YearsCode','Age']])
df_age_calc['learned_code'] = (cleaned_df['Age'] - cleaned_df['YearsCode'])
less_than_18 = df_age_calc[ df_age_calc['learned_code'] < 15].index
less_over_65 = df_age_calc[ df_age_calc['Age'] > 65].index
df_age_calc.drop(less_than_18 , inplace=True)
df_age_calc.drop(less_over_65 , inplace=True)
df_age_calc = df_age_calc.drop(columns=['YearsCode','Age'])

df_language = pd.get_dummies(cleaned_df['LanguageWorkedWith'].apply(pd.Series).stack()).sum(level=0)
df_language['LanguageWorkedWith_Total'] = df_language.sum(axis=1)

df_ethnicity = pd.get_dummies(cleaned_df['Ethnicity'].apply(pd.Series).stack()).sum(level=0)

cleaned_df = cleaned_df.join(df_age_calc.set_index('Respondent'), on='Respondent')
cleaned_df = cleaned_df.join(df_language)
cleaned_df = cleaned_df.join(df_ethnicity)

cleaned_df = cleaned_df.drop(labels='LanguageWorkedWith',axis=1)
cleaned_df = cleaned_df.drop(labels='Ethnicity',axis=1)

cleaned_df = pd.get_dummies(cleaned_df)

In [11]:
# Cleaning ConvertedComp column
# df_converted_comp = pd.DataFrame(cleaned_df[['Respondent', 'YearsCode', 'Age']])
# troll = df_converted_comp[ (df_converted_comp['YearsCode'] < 10) ].index
troll = cleaned_df[ (cleaned_df['YearsCode'] < 10) & (cleaned_df['ConvertedComp'] >= 1000000)].index
cleaned_df.drop(troll, inplace=True)
troll_2 = cleaned_df[cleaned_df['ConvertedComp'] == 2000000].index
cleaned_df.drop(troll_2, inplace=True)
troll_3 = cleaned_df[cleaned_df['ConvertedComp'] == 1000000].index
cleaned_df.drop(troll_3, inplace=True)

In [12]:
df = cleaned_df.dropna()
df

,Respondent,ConvertedComp,WorkWeekHrs,YearsCode,Age,CodeRevHrs,YearsCodePro,learned_code,Assembly,Bash/Shell/PowerShell,...,SOPartFreq_Less than once per month or monthly,SOPartFreq_Multiple times per day,FizzBuzz_No,FizzBuzz_Yes,Continent_Africa,Continent_Asia,Continent_Europe,Continent_North America,Continent_Oceania Australia,Continent_South America
13,14,57060.0,40.0,13,31.0,4.0,2,18.0,0,0,...,1,0,1,0,0,0,1,0,0,0
16,17,65277.0,45.0,5,29.0,5.0,2,24.0,0,1,...,0,0,1,0,0,0,0,0,1,0
18,19,31140.0,8.0,14,31.0,3.0,13,17.0,0,0,...,0,0,1,0,0,0,0,0,0,1
22,23,69000.0,40.0,3,22.0,8.0,1,19.0,0,1,...,0,0,1,0,0,0,0,1,0,0
28,29,100000.0,40.0,4,32.0,2.0,2,28.0,0,1,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88300,88852,3000.0,84.0,4,31.0,2.0,3,27.0,0,0,...,1,0,1,0,1,0,0,0,0,0
88301,88853,110000.0,36.0,8,27.0,4.0,6,19.0,0,1,...,0,0,0,1,0,0,0,1,0,0
88315,88868,797436.0,35.0,15,30.0,5.0,7,15.0,0,0,...,0,1,0,1,0,0,1,0,0,0
88323,88876,180000.0,40.0,8,23.0,3.0,2,15.0,0,1,...,1,0,1,0,0,0,0,1,0,0


In [13]:
list(df.columns)

['Respondent',
 'ConvertedComp',
 'WorkWeekHrs',
 'YearsCode',
 'Age',
 'CodeRevHrs',
 'YearsCodePro',
 'learned_code',
 'Assembly',
 'Bash/Shell/PowerShell',
 'C',
 'C#',
 'C++',
 'Clojure',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'HTML/CSS',
 'Java',
 'JavaScript',
 'Kotlin',
 'Objective-C',
 'Other(s):',
 'PHP',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VBA',
 'WebAssembly',
 'LanguageWorkedWith_Total',
 'Biracial',
 'Black or of African descent',
 'East Asian',
 'Hispanic or Latino/Latina',
 'Middle Eastern',
 'Multiracial',
 'Native American, Pacific Islander, or Indigenous Australian',
 'South Asian',
 'White or of European descent',
 'OrgSize_1,000 to 4,999 employees',
 'OrgSize_10 to 19 employees',
 'OrgSize_10,000 or more employees',
 'OrgSize_100 to 499 employees',
 'OrgSize_2-9 employees',
 'OrgSize_20 to 99 employees',
 'OrgSize_5,000 to 9,999 employees',
 'OrgSize_500 to 999 employees',
 'OrgSize_Just me - I am a freelancer, sole

## Feature Engineering


In [14]:
def add_interaction(data,column1,column2):
    data['{}:{}'.format(column1,column2)] = data.apply(lambda x: x[column1]*x[column2],axis=1)

In [15]:
#df['gender:dependents']= df.apply(lambda x: x['Gender_Woman']*x['Dependents_Yes'],axis=1)
add_interaction(df,'Gender_Woman','Dependents_Yes')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
add_interaction(df,'White or of European descent','Gender_Man')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
list(df.columns)

['Respondent',
 'ConvertedComp',
 'WorkWeekHrs',
 'YearsCode',
 'Age',
 'CodeRevHrs',
 'YearsCodePro',
 'learned_code',
 'Assembly',
 'Bash/Shell/PowerShell',
 'C',
 'C#',
 'C++',
 'Clojure',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'HTML/CSS',
 'Java',
 'JavaScript',
 'Kotlin',
 'Objective-C',
 'Other(s):',
 'PHP',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VBA',
 'WebAssembly',
 'LanguageWorkedWith_Total',
 'Biracial',
 'Black or of African descent',
 'East Asian',
 'Hispanic or Latino/Latina',
 'Middle Eastern',
 'Multiracial',
 'Native American, Pacific Islander, or Indigenous Australian',
 'South Asian',
 'White or of European descent',
 'OrgSize_1,000 to 4,999 employees',
 'OrgSize_10 to 19 employees',
 'OrgSize_10,000 or more employees',
 'OrgSize_100 to 499 employees',
 'OrgSize_2-9 employees',
 'OrgSize_20 to 99 employees',
 'OrgSize_5,000 to 9,999 employees',
 'OrgSize_500 to 999 employees',
 'OrgSize_Just me - I am a freelancer, sole

In [18]:
ethnicities = ['Biracial',
 'Black or of African descent',
 'East Asian',
 'Hispanic or Latino/Latina',
 'Middle Eastern',
 'Multiracial',
 'Native American, Pacific Islander, or Indigenous Australian',
 'South Asian']

In [19]:
for ethnicity in ethnicities:
    add_interaction(df,ethnicity,'Gender_Woman')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
# extraversion = ['Extraversion_In real life (in person)',
#  'Extraversion_Neither',
#  'Extraversion_Online']

# for quality in extraversion:
#     add_interaction(df,quality,'Gender_Woman')

In [21]:
continents =  ['Continent_Africa',
 'Continent_Asia',
 'Continent_Europe',
 'Continent_North America',
 'Continent_Oceania Australia',
 'Continent_South America']

In [22]:
# for continent in continents:
#     add_interaction(df,continent,'Gender_Woman')

In [23]:
add_interaction(df,'ImpSyn_Far above average','Gender_Man')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
list(df.columns)

['Respondent',
 'ConvertedComp',
 'WorkWeekHrs',
 'YearsCode',
 'Age',
 'CodeRevHrs',
 'YearsCodePro',
 'learned_code',
 'Assembly',
 'Bash/Shell/PowerShell',
 'C',
 'C#',
 'C++',
 'Clojure',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'HTML/CSS',
 'Java',
 'JavaScript',
 'Kotlin',
 'Objective-C',
 'Other(s):',
 'PHP',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VBA',
 'WebAssembly',
 'LanguageWorkedWith_Total',
 'Biracial',
 'Black or of African descent',
 'East Asian',
 'Hispanic or Latino/Latina',
 'Middle Eastern',
 'Multiracial',
 'Native American, Pacific Islander, or Indigenous Australian',
 'South Asian',
 'White or of European descent',
 'OrgSize_1,000 to 4,999 employees',
 'OrgSize_10 to 19 employees',
 'OrgSize_10,000 or more employees',
 'OrgSize_100 to 499 employees',
 'OrgSize_2-9 employees',
 'OrgSize_20 to 99 employees',
 'OrgSize_5,000 to 9,999 employees',
 'OrgSize_500 to 999 employees',
 'OrgSize_Just me - I am a freelancer, sole

In [25]:
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import sklearn.preprocessing as preprocessing

In [26]:
import statsmodels.api as sm

In [27]:
def sm_OLS(x_values,y_values):
    x_values = sm.add_constant(x_values)
    model = sm.OLS(y_values,x_values)
    results = model.fit()
    results_summary = results.summary()
    results_as_html = results_summary.tables[1].as_html()

    model_performance = {}
    model_performance['MSE'] = results.mse_model
    model_performance['MSE_resid'] = results.mse_resid
    model_performance['RMSE'] = results.mse_resid**.5
    model_performance['R^2'] = results.rsquared
    model_performance['R^2_adjusted'] = results.rsquared_adj
    model_performance['F-stat'] = results.fvalue
    model_performance['AIC'] = results.aic
    model_performance['BIC'] = results.bic
    
    model_performance = pd.DataFrame.from_dict(model_performance,orient='index',columns=['Value'])
    
    #print(results.summary())
    variables_as_html = results_summary.tables[1].as_html()
    variables = pd.read_html(results_as_html, header=0, index_col=0)[0]
    
    outcomes = {}
    outcomes['Observed_values'] = y_values
    outcomes['Predicted_values'] = results.predict()
    outcomes['Residuals'] = results.resid
    
    #outcomes['Conf']
    
    outcomes = pd.DataFrame(outcomes)
    #print(results.summary())
    return model_performance,variables,outcomes

In [28]:
X = df.drop(columns=['Respondent','ConvertedComp'])
Y = df['ConvertedComp']

sm_performance,sm_variables,sm_outcomes = sm_OLS(X,Y)
#sm_performance['Value'] = sm_performance['Value'].apply(lambda x: round(x,10))

//anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [29]:
pd.set_option('display.max_rows',2000)

In [30]:
sm_performance.transpose()

,MSE,MSE_resid,RMSE,R^2,R^2_adjusted,F-stat,AIC,BIC
Value,5.495778e+11,1.806648e+10,134411.619227,0.270342,0.261455,30.419744,574100.177922,576192.156931


In [31]:
sm_variables

,coef,std err,t,P>|t|,[0.025,0.975]
const,-3.578385e+03,2.003781e+03,-1.786,0.074,-7.505945e+03,3.491750e+02
WorkWeekHrs,4.626825e+02,1.118320e+02,4.137,0.000,2.434840e+02,6.818810e+02
YearsCode,3.225449e+02,2.090350e+02,1.543,0.123,-8.717900e+01,7.322690e+02
Age,4.010050e+01,1.755050e+02,0.228,0.819,-3.039030e+02,3.841040e+02
CodeRevHrs,3.410805e+02,2.221830e+02,1.535,0.125,-9.441500e+01,7.765760e+02
YearsCodePro,2.299615e+03,3.621300e+02,6.350,0.000,1.589814e+03,3.009416e+03
learned_code,-2.824444e+02,1.606100e+02,-1.759,0.079,-5.972520e+02,3.236300e+01
Assembly,-1.780880e+03,5.400478e+03,-0.330,0.742,-1.240000e+04,8.804461e+03
Bash/Shell/PowerShell,3.572280e+03,2.185009e+03,1.635,0.102,-7.105010e+02,7.855062e+03
C,-1.785884e+03,3.433037e+03,-0.520,0.603,-8.514893e+03,4.943125e+03


In [32]:
sm_outcomes

,Observed_values,Predicted_values,Residuals
13,57060.0,220352.540184,-163292.540184
16,65277.0,63428.157398,1848.842602
18,31140.0,46612.596706,-15472.596706
22,69000.0,152894.874425,-83894.874425
28,100000.0,239381.835184,-139381.835184
...,...,...,...
88300,3000.0,77811.026910,-74811.026910
88301,110000.0,91266.363391,18733.636609
88315,797436.0,187853.694791,609582.305209
88323,180000.0,167914.647185,12085.352815


In [33]:
sm_variables.sort_values('coef',ascending=False)[:25]

,coef,std err,t,P>|t|,[0.025,0.975]
Country_Liechtenstein,713900.0,132000.000,5.427,0.000,456000.000,972000.0
Country_Kuwait,397800.0,93500.000,4.254,0.000,215000.000,581000.0
Country_Republic of Korea,352600.0,66500.000,5.300,0.000,222000.000,483000.0
Country_Ireland,181500.0,12100.000,15.035,0.000,158000.000,205000.0
Country_United States,177400.0,15400.000,11.553,0.000,147000.000,207000.0
Country_United Kingdom,136600.0,5979.988,22.843,0.000,125000.000,148000.0
Country_South Korea,132200.0,26900.000,4.912,0.000,79500.000,185000.0
Country_Canada,126000.0,15900.000,7.943,0.000,94900.000,157000.0
Country_Norway,121100.0,12400.000,9.748,0.000,96800.000,145000.0
Country_Somalia,108200.0,131000.000,0.828,0.408,-148000.000,365000.0


In [34]:
df

,Respondent,ConvertedComp,WorkWeekHrs,YearsCode,Age,CodeRevHrs,YearsCodePro,learned_code,Assembly,Bash/Shell/PowerShell,...,White or of European descent:Gender_Man,Biracial:Gender_Woman,Black or of African descent:Gender_Woman,East Asian:Gender_Woman,Hispanic or Latino/Latina:Gender_Woman,Middle Eastern:Gender_Woman,Multiracial:Gender_Woman,"Native American, Pacific Islander, or Indigenous Australian:Gender_Woman",South Asian:Gender_Woman,ImpSyn_Far above average:Gender_Man
13,14,57060.0,40.0,13,31.0,4.0,2,18.0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,17,65277.0,45.0,5,29.0,5.0,2,24.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,19,31140.0,8.0,14,31.0,3.0,13,17.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22,23,69000.0,40.0,3,22.0,8.0,1,19.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,29,100000.0,40.0,4,32.0,2.0,2,28.0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88300,88852,3000.0,84.0,4,31.0,2.0,3,27.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88301,88853,110000.0,36.0,8,27.0,4.0,6,19.0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
88315,88868,797436.0,35.0,15,30.0,5.0,7,15.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88323,88876,180000.0,40.0,8,23.0,3.0,2,15.0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
languages = ['Assembly',
 'Bash/Shell/PowerShell',
 'C',
 'C#',
 'C++',
 'Clojure',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'HTML/CSS',
 'Java',
 'JavaScript',
 'Kotlin',
 'Objective-C',
 'Other(s):',
 'PHP',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VBA',
 'WebAssembly']

In [36]:
list(df.columns)

['Respondent',
 'ConvertedComp',
 'WorkWeekHrs',
 'YearsCode',
 'Age',
 'CodeRevHrs',
 'YearsCodePro',
 'learned_code',
 'Assembly',
 'Bash/Shell/PowerShell',
 'C',
 'C#',
 'C++',
 'Clojure',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'HTML/CSS',
 'Java',
 'JavaScript',
 'Kotlin',
 'Objective-C',
 'Other(s):',
 'PHP',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VBA',
 'WebAssembly',
 'LanguageWorkedWith_Total',
 'Biracial',
 'Black or of African descent',
 'East Asian',
 'Hispanic or Latino/Latina',
 'Middle Eastern',
 'Multiracial',
 'Native American, Pacific Islander, or Indigenous Australian',
 'South Asian',
 'White or of European descent',
 'OrgSize_1,000 to 4,999 employees',
 'OrgSize_10 to 19 employees',
 'OrgSize_10,000 or more employees',
 'OrgSize_100 to 499 employees',
 'OrgSize_2-9 employees',
 'OrgSize_20 to 99 employees',
 'OrgSize_5,000 to 9,999 employees',
 'OrgSize_500 to 999 employees',
 'OrgSize_Just me - I am a freelancer, sole

In [37]:
import charlie_functions

In [38]:
a,b,c = charlie_functions.sm_OLS(X,Y)

In [39]:
a

,Value
MSE,5.495778e+11
MSE_resid,1.806648e+10
RMSE,1.344116e+05
R^2,2.703420e-01
R^2_adjusted,2.614549e-01
F-stat,3.041974e+01
AIC,5.741002e+05
BIC,5.761922e+05


In [40]:
b

,coef,std err,t,P>|t|,[0.025,0.975]
const,-3.578385e+03,2.003781e+03,-1.786,0.074,-7.505945e+03,3.491750e+02
WorkWeekHrs,4.626825e+02,1.118320e+02,4.137,0.000,2.434840e+02,6.818810e+02
YearsCode,3.225449e+02,2.090350e+02,1.543,0.123,-8.717900e+01,7.322690e+02
Age,4.010050e+01,1.755050e+02,0.228,0.819,-3.039030e+02,3.841040e+02
CodeRevHrs,3.410805e+02,2.221830e+02,1.535,0.125,-9.441500e+01,7.765760e+02
YearsCodePro,2.299615e+03,3.621300e+02,6.350,0.000,1.589814e+03,3.009416e+03
learned_code,-2.824444e+02,1.606100e+02,-1.759,0.079,-5.972520e+02,3.236300e+01
Assembly,-1.780880e+03,5.400478e+03,-0.330,0.742,-1.240000e+04,8.804461e+03
Bash/Shell/PowerShell,3.572280e+03,2.185009e+03,1.635,0.102,-7.105010e+02,7.855062e+03
C,-1.785884e+03,3.433037e+03,-0.520,0.603,-8.514893e+03,4.943125e+03


In [41]:
c

,Observed_values,Predicted_values,Residuals
13,57060.0,220352.540184,-163292.540184
16,65277.0,63428.157398,1848.842602
18,31140.0,46612.596706,-15472.596706
22,69000.0,152894.874425,-83894.874425
28,100000.0,239381.835184,-139381.835184
...,...,...,...
88300,3000.0,77811.026910,-74811.026910
88301,110000.0,91266.363391,18733.636609
88315,797436.0,187853.694791,609582.305209
88323,180000.0,167914.647185,12085.352815


In [42]:
regr = linear_model.LinearRegression()